Here, the performance of the lookup tables optimized in the "example_F_lookup_training_test.ipynb" on state stabilization for longer time durations can be tested.

In [1]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from helpers import (
    transport_unitary,
    initialize_chain_of_zeros,
    partial_trace,
    generate_random_state,
    generate_random_initial_state,
    test_implementations,
)
import numpy as np
from feedback_grape.utils.fidelity import fidelity
from feedback_grape.utils.operators import sigmam
from library.utils.qubit_chain_1D import embed
from feedback_grape.utils.solver import mesolve
from tqdm import tqdm
import jax
import jax.numpy as jnp

test_implementations()

In [2]:
# Physical parameters
n = 2 # number of qubits per chain (>= 3)
N_chains = 2 # Number of parallel chains to simulate
gamma = 0.25 # Decay constant
evaluation_time_steps = 200 # Number of time steps for evaluation
batch_size = 256 # Number of random states to evaluate in parallel

# Path to save results
results_path = "./evaluation_results/baseline_fidelities.npz"

In [3]:
base_dim = 2**N_chains
T_half = transport_unitary(0.5, n, N_chains)
c_ops = [sum([gamma * embed(sigmam(), idx, N_chains*n) for idx in range(N_chains*n)])]

# Evolve all basis states and compute fidelities
keys = jax.random.split(jax.random.PRNGKey(42), batch_size)
states = jnp.array([generate_random_initial_state(key, N_chains) for key in keys])
target_states = jnp.array([generate_random_state(key, N_chains) for key in keys])

fidelities_each = np.zeros((len(states), evaluation_time_steps+1))
for i, (state, target_state) in enumerate(zip(states, target_states)):
    fidelities_each[i, 0] = fidelity(
        C_target=target_state,
        U_final=state,
        evo_type="density",
    )

def propagate_single_timestep(rho, rho_target):
    tmp = initialize_chain_of_zeros(rho, n=n, N_chains=N_chains)
    tmp = T_half @ tmp @ T_half.conj().T

    tmp = mesolve(
        jump_ops=c_ops,
        rho0=tmp,
    )

    tmp = T_half @ tmp @ T_half.conj().T
    tmp = partial_trace(tmp, sys_A_dim=base_dim**(n-1), sys_B_dim=base_dim)

    fid = fidelity(C_target=rho_target, U_final=tmp, evo_type="density")

    return tmp, fid

propagate_single_timestep_vmap = jax.vmap(jax.jit(propagate_single_timestep))

for i in tqdm(range(evaluation_time_steps)):
    states, fid = propagate_single_timestep_vmap(states, target_states)
    fidelities_each[:, i+1] = fid

    #for j, rho in enumerate(states):
    #    assert np.all(np.isclose(rho, rho.conj().T)), "State is not Hermitian"
    #    assert np.isclose(np.trace(rho).real, 1.0), "State is not normalized"
    #    assert np.all(np.linalg.eigvalsh(rho) >= -1e-10), "State is not positive semidefinite"

fidelities_each = fidelities_each.mean(axis=0) # Average over all basis states

# Reshape back to matrix form
np.savez(results_path, fidelities=fidelities_each)

100%|██████████| 200/200 [00:09<00:00, 20.17it/s]


In [4]:
fidelities_each

array([0.64235275, 0.61621116, 0.5911702 , 0.56727525, 0.54454959,
       0.52299869, 0.50261382, 0.48337509, 0.46525401, 0.44821555,
       0.43221992, 0.417224  , 0.40318253, 0.39004903, 0.37777662,
       0.36631862, 0.35562904, 0.34566293, 0.33637673, 0.32772841,
       0.31967767, 0.31218601, 0.30521682, 0.29873536, 0.29270879,
       0.28710614, 0.28189825, 0.27705775, 0.27255897, 0.26837788,
       0.26449202, 0.26088042, 0.25752352, 0.25440311, 0.25150222,
       0.24880508, 0.24629703, 0.24396444, 0.24179468, 0.23977599,
       0.23789751, 0.23614913, 0.23452149, 0.23300591, 0.23159437,
       0.23027941, 0.22905412, 0.22791212, 0.22684748, 0.22585472,
       0.22492875, 0.22406486, 0.22325869, 0.2225062 , 0.22180363,
       0.22114751, 0.22053462, 0.21996198, 0.21942681, 0.21892654,
       0.2184588 , 0.21802135, 0.21761216, 0.21722931, 0.21687102,
       0.21653566, 0.21622169, 0.21592768, 0.21565232, 0.21539437,
       0.21515269, 0.2149262 , 0.21471393, 0.21451493, 0.21432